Dependencies and imports

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab import userdata
# import json
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.metrics import R2Score
import keras.backend as K
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from statistics import stdev
import pandas_datareader as dr
from datetime import date
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns


don't install this package unless you want to save the charts

In [ ]:
!sudo apt install texlive-latex-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libfontenc1 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1
  libruby3.0 libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1
  libzzip-0-13 lmodern poppler-data preview-latex-style rake ruby
  ruby-net-telnet ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0
  rubygems-integration t1utils tex-common tex-gyre texlive-base
  texlive-binaries texlive-fonts-recommended texlive-latex-base
  texlive-latex-recommended texlive-pictures texlive-plain-generic tipa
  xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-framework-java
  libcommons-logging

In [ ]:
# define date range for stock
start_date = "1950-01-01"
curr_date = date.today()

# number of days used to make prediction
trail = 730 # 2 years

# define neural network model
def model_regression():
    m = Sequential()
    m.add(LSTM(units=8, input_shape=(trail, 1)))
    m.add(Dense(units=8, activation='gelu'))
    m.add(Dense(units=1))
    return m

In [ ]:
# Fetch the data from Yahoo Finance
df = yf.download('^GSPC', start_date, curr_date)

raw_data = df.filter(['Close'])

raw_data = raw_data.replace(',', '', regex=True)

# normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = pd.DataFrame(scaler.fit_transform(raw_data))
scaled_data = scaled_data.rename(columns={0: 'Close'})

# use 75% of data to train model, 25% to test
train_data_size = int(0.75 * len(raw_data))
train_data = scaled_data[:train_data_size]

# subtract tail since its is needed to predict 1st day of test data
test_data = scaled_data[train_data_size-trail:]
test_data = test_data.rename(columns={0: 'Close'})

# convert train and test data to numpy arrays
train_data_array = np.array(train_data)
test_data_array = np.array(test_data)

# split train data into inputs and targets
x_train, y_train = [], []
for i in range(train_data_array.shape[0] - trail):
  x_train.append(train_data_array[i:i + trail])    # input will be closing prices from tail days
  y_train.append(train_data_array[i + trail, 0])   # target will be closing price one day after

x_train, y_train = np.array(x_train), np.array(y_train)

# split test data into inputs and targets
x_test, y_test = [], []
for i in range(test_data_array.shape[0] - trail):
  x_test.append(test_data_array[i:i + trail])
  y_test.append(test_data_array[i + trail])

# convert to numpy arrays to be read for training
x_test, y_test = np.array(x_test), np.array(y_test)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
batch_sizes = [train_data_size, 1024, 512, 128, 64, 32, 1]

# load optimization algorithms
def fetch_algs():
  plain = SGD(learning_rate = 0.001, momentum = 0.0, name = 'Plain')
  momentum = SGD(learning_rate = 0.001, momentum=0.9, name = 'Momentum')
  rmsprop = RMSprop(learning_rate = 0.001, name = 'RMSprop')
  adam = Adam(learning_rate = 0.001, name = 'Adam')

  return [plain, momentum, rmsprop, adam]


In [ ]:
for size in batch_sizes:
  opt_algs = fetch_algs()
  for alg in opt_algs:
    print(f'==================== BATCH SIZE = {size}, OPT_ALG = {alg.name} ====================\n\n')
    model = model_regression()
    model.compile(
        loss='log_cosh',
        optimizer=alg,
        metrics=['mean_squared_error','mean_absolute_error', 'mean_absolute_percentage_error']
        )
    model.summary()

    history = model.fit(x_train, y_train, batch_size = size, epochs = 30)

    loss_stdev = stdev(history.history['loss'])
    mse_stdev = stdev(history.history['mean_squared_error'])
    mae_stdev = stdev(history.history['mean_absolute_error'])
    mape_stdev = stdev(history.history['mean_absolute_percentage_error'])
    print(f'\n\nLoss St.Dev: {loss_stdev} - MSE St.Dev: {mse_stdev} - MAE Std.Dev: {mae_stdev} - MAPE St.Dev: {mape_stdev}\n\n')

    # unscale the predictions to they can be inserted into to the test data
    scaled_predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(scaled_predictions)
    actual = scaler.inverse_transform(y_test)

    residuals = actual - predictions

    plt.style.use('seaborn')
    # 1. Training & Validation Loss Plot
    plt.figure(figsize=(6, 4))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.title(f'Training Loss ({alg.name}, Batch Size = {size})')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig(f'P1-BS{size}-{alg.name}.pgf')

    # 2. Actual vs Predicted - Line Chart
    plt.figure(figsize=(6, 4))
    plt.plot(actual, label='Actual Values', linestyle='-', marker='o', color='b')
    plt.plot(predictions, label='Predictions', linestyle='-', marker='x', color='r')
    plt.title(f'Actual vs. Predicted Values ({alg.name}, Batch Size = {size})')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.savefig(f'P2-BS{size}-{alg.name}.pgf')
    plt.show()

    # 3. Actual vs Predicted - Scatter Plot
    plt.figure(figsize=(6, 4))
    plt.scatter(actual, predictions, alpha=0.5, color='purple')
    plt.title(f'Correlation between Actual and Predictions ({alg.name}, Batch Size = {size})')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.plot([min(actual), max(actual)], [min(actual), max(actual)], 'k--', lw=2)  # Line of perfect fit
    plt.savefig(f'P3-BS{size}-{alg.name}.pgf')
    plt.show()

    # 4. Histogram of Residuals
    residuals = actual.flatten() - predictions.flatten()
    plt.figure(figsize=(6, 4))
    plt.hist(residuals, bins=30, color='gray', alpha=0.7)
    plt.title(f'Histogram of Prediction Residuals ({alg.name}, Batch Size = {size})')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'P4-BS{size}-{alg.name}.pgf')
    plt.show()

    print(f'==================== ==================== ==================== ==================== ==================== ==================== ====================\n')
    print(f'==================== ==================== ==================== ==================== ==================== ==================== ====================\n\n')



==================== BATCH SIZE = 14022, OPT_ALG = Plain ====================


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 8)                 320       
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
1/1 [==============================] - 8s 8s/step - loss: 0.0042 - mean_squared_error: 0.0084 - mean_absolute_error: 0.0570 - mean_absolute_percentage_error: 96.7210
Epoch 2/30
1/1 [=================

<ipython-input-6-c8dce139631b>:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


==================== ==================== ==================== ==================== ==================== ==================== ====================

==================== ==================== ==================== ==================== ==================== ==================== ====================


==================== BATCH SIZE = 14022, OPT_ALG = Momentum ====================


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 8)                 320       
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-traina

<ipython-input-6-c8dce139631b>:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


==================== ==================== ==================== ==================== ==================== ==================== ====================

==================== ==================== ==================== ==================== ==================== ==================== ====================


==================== BATCH SIZE = 14022, OPT_ALG = RMSprop ====================


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 8)                 320       
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainab

...

==================== ==================== ==================== ==================== ==================== ==================== ====================

==================== ==================== ==================== ==================== ==================== ==================== ====================


==================== BATCH SIZE = 128, OPT_ALG = Adam ====================


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 8)                 320       
                                                                 
 dense_30 (Dense)            (None, 8)                 72        
                                                                 
 dense_31 (Dense)            (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainable p

<ipython-input-6-c8dce139631b>:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


==================== ==================== ==================== ==================== ==================== ==================== ====================

==================== ==================== ==================== ==================== ==================== ==================== ====================


==================== BATCH SIZE = 64, OPT_ALG = Plain ====================


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 8)                 320       
                                                                 
 dense_32 (Dense)            (None, 8)                 72        
                                                                 
 dense_33 (Dense)            (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainable p

<ipython-input-6-c8dce139631b>:28: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


==================== ==================== ==================== ==================== ==================== ==================== ====================

==================== ==================== ==================== ==================== ==================== ==================== ====================


==================== BATCH SIZE = 64, OPT_ALG = Momentum ====================


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_17 (LSTM)              (None, 8)                 320       
                                                                 
 dense_34 (Dense)            (None, 8)                 72        
                                                                 
 dense_35 (Dense)            (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainabl